## 1. Setup Environment

In [ ]:
# Clone the repository
!git clone https://github.com/Snaju003/Final-year-project.git
%cd Final-year-project

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Check GPU availability
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Using CPU")

## 2. Import Libraries

In [ ]:
import sys
import os
sys.path.insert(0, '/content/Final-year-project')

import torch
import torch.nn as nn
from pathlib import Path
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# Import project modules
from src.models import build_all_models
from src.inference import EnsembleModel

## 3. Load Model and Run Inference

In [ ]:
# Load the ensemble model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnsembleModel().to(device)

# Load pre-trained weights if available
model_path = 'models/ensemble/ensemble_final.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ Model loaded from {model_path}")
else:
    print(f"⚠️ Model weights not found. Using random initialization.")

model.eval()
print(f"✅ Model ready on {device}")

In [ ]:
# Function to predict on a single image
def predict_image(image_path, model, device):
    """
    Predict deepfake probability for a single image
    
    Args:
        image_path: Path to image file
        model: Ensemble model
        device: torch device
    
    Returns:
        deepfake_probability: Float between 0 and 1
    """
    from torchvision import transforms
    
    # Image preprocessing
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    # Inference
    with torch.no_grad():
        output = model(image)
    
    return output.item()

# Example usage (uncomment when you have an image)
# image_path = 'your_image.jpg'
# prob = predict_image(image_path, model, device)
# print(f"Deepfake probability: {prob:.4f}")

## 4. Batch Processing

In [ ]:
def batch_predict(image_dir, model, device):
    """
    Predict on multiple images in a directory
    """
    from torchvision import transforms
    from pathlib import Path
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    results = {}
    image_paths = list(Path(image_dir).glob('*.jpg')) + list(Path(image_dir).glob('*.png'))
    
    for image_path in tqdm(image_paths, desc="Processing images"):
        try:
            image = Image.open(image_path).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = model(image)
            
            results[image_path.name] = output.item()
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    
    return results

# Example usage
# results = batch_predict('path/to/images', model, device)
# for img_name, prob in results.items():
#     print(f"{img_name}: {prob:.4f}")

## 5. Download Results

In [ ]:
# Download results to local machine
from google.colab import files

# Save results to a file
import json
# results_path = 'results.json'
# with open(results_path, 'w') as f:
#     json.dump(results, f, indent=2)

# Download
# files.download(results_path)